In [ ]:
#!/usr/bin/python
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy import odr
import pylab as py
from matplotlib import gridspec

from redTools import *
from Kcorrect import *
################################################################# 
def add_axis(ax, xlim, ylim):
    
    x1, x2 = xlim[0], xlim[1]
    y1, y2 = ylim[0], ylim[1]
    ax.set_xlim(x1, x2)
    ax.set_ylim(y1, y2)

    ax.minorticks_on()
    ax.tick_params(which='major', length=5, width=1.0)
    ax.tick_params(which='minor', length=2, color='#000033', width=1.0)     
    
    # additional Y-axis (on the right)
    y_ax = ax.twinx()
    y_ax.set_ylim(y1, y2)
    y_ax.set_yticklabels([])
    y_ax.minorticks_on()
    y_ax.tick_params(which='major', length=5, width=1.0, direction='in')
    y_ax.tick_params(which='minor', length=2, color='#000033', width=1.0, direction='in')

    # additional X-axis (on the top)
    x_ax = ax.twiny()
    x_ax.set_xlim(x1, x2)
    x_ax.set_xticklabels([])
    x_ax.minorticks_on()
    x_ax.tick_params(which='major', length=5, width=1.0, direction='in')
    x_ax.tick_params(which='minor', length=2, color='#000033', width=1.0, direction='in')
    

########################################################### Begin

inFile  = 'ESN_HI_catal.csv'
table   = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None)

table = extinctionCorrect(table)
table = Kcorrection(table)

band1 = 'i'
band2 = 'w2'

if band2=='w1':
    text1 = band1+'-W1'            # example: cr-W1
    text2 = '$c21W_1$'              # example: c21w
else: 
    text1 = band1+'-W2' 
    text2 = '$c21W_2$'

delta = np.abs(table[band2]-table[band2+'_'])
index, = np.where(delta<=0.15)
table = trim(table, index)

delta = np.abs(table[band1]-table[band1+'_'])
index, = np.where(delta<=0.15)
table = trim(table, index)

table['c21w'] = table['m21'] - table[band2]
table['r_w1'] = table[band1] - table[band2]

table['Ec21w'] = np.sqrt(table['m21_e']**2+0.05**2)
table['Er_w1'] = 0.*table['r_w1']+0.1

index, = np.where(table['logWimx']>1)
table = trim(table, index)

index, = np.where(table['r_w1']<4)
table = trim(table, index)

index, = np.where(table['c21w']<10)
table = trim(table, index)


########################################################Face-ON

## Get the initial estimations using Face-on galaxies
## AB:    a0*logWimx+b0
## Delta: alfa*X**2+beta*X+gama
AB, Delta, table, cov, Delta_e = faceON(table)
########################################################### END


pgc = table['pgc']
logWimx = table['logWimx']
logWimx_e = table['logWimx_e']
inc = table['inc']
r_w1 = table['r_w1']
c21w = table['c21w'] 
Er_w1 = table['Er_w1']
Ec21w = table['Ec21w']


C82  = table['C82_w2']   # concentration 80%/20%
mu50 = table['w2']+2.5*np.log10(2.*np.pi*(table['R50_w2']*60)**2)

a0, b0 = AB[0],AB[1]
a = 1./a0
b = -1.*b0/a0

#########################################################################  

fig = py.figure(figsize=(9, 4), dpi=100)   
fig.subplots_adjust(wspace=0.25, top=0.95, bottom=0.13, left=0.09, right=0.98)
gs = gridspec.GridSpec(1, 2) 

ax = plt.subplot(gs[0]) 
ax.plot([-1,4], [-a+b,a*4+b], 'k--')

for i in range(len(pgc)):
   if c21w[i]<1  :
       p1, = ax.plot(r_w1[i], logWimx[i], 'b.', markersize=5, alpha=1, label=r"$c21W_1 < 1$")
       #plt.errorbar(r_w1[i], logWimx[i], xerr=Er_w1[i], yerr=logWimx_e[i], color='b', alpha=0.2)
   if c21w[i]>=1 and c21w[i]< 2:
       p2, = ax.plot(r_w1[i], logWimx[i], 'g.', markersize=5, alpha=1, label=r"$1 < c21W_1 < 3$")  
       #plt.errorbar(r_w1[i], logWimx[i], xerr=Er_w1[i], yerr=logWimx_e[i], color='g', alpha=0.2)
   if c21w[i]>=2 and c21w[i]< 3:
       ax.plot(r_w1[i], logWimx[i], '.', color='green', markersize=5, alpha=1)   
       #plt.errorbar(r_w1[i], logWimx[i], xerr=Er_w1[i], yerr=logWimx_e[i], color='g', alpha=0.2)
   if c21w[i]>=3:
       p3, = ax.plot(r_w1[i], logWimx[i], 'r.', markersize=5, alpha=1, label=r"$3 < c21W_1$")   
       #plt.errorbar(r_w1[i], logWimx[i], xerr=Er_w1[i], yerr=logWimx_e[i], color='r', alpha=0.2)
       
ax.set_xlabel('$'+text1+'$', fontsize=14)
ax.set_ylabel(r'$Log( W_{mx}^i)$', fontsize=14)

rw_lim = [-2.5,1.5]
if band1=='u': rw_lim = [-0.5,3.5]
if band1=='z': rw_lim = [-3.0,1.0]
if band1=='g': rw_lim = [-2.0,2.0]

add_axis(ax,rw_lim,[1.7,3])
  
# Legend
lns = [p1, p2, p3]
ax.legend(handles=lns, loc=0, fontsize=11)

X_err_median = np.median(Er_w1[i])
Y_err_median = np.median(logWimx_e[i])
Ylm = ax.get_ylim() ; Xlm = ax.get_xlim()
x0 = 0.1*Xlm[0]+0.9*Xlm[1]
y0 = 0.2*Ylm[0]+0.8*Ylm[1]
plt.errorbar([x0], [y0], xerr=[Er_w1[i]], yerr=[Y_err_median], color='k', fmt='o', alpha=0.7, capsize=3, markersize=5)
       
#########################################################################  

ax = plt.subplot(gs[1])

delta = r_w1-(a0*logWimx+b0)
ax.plot(c21w, delta, '.', color='black', markersize=3, alpha=0.5)
#ax.errorbar(c21w, delta, xerr=Ec21w, yerr=delta*0.+0.1, color='k', fmt='.', alpha=0.2)

add_axis(ax,[-2,8],[-1.5,1.5])

for i in range(-1,6):
    
    x = []
    y = []
    for ii in range(len(c21w)):
        xi = c21w[ii]
        if xi>=i and xi<i+1:
            x.append(xi)
            y.append(delta[ii])
    if len(x)>0:
        ax.errorbar(np.median(x), np.median(y), yerr=np.std(y), xerr=np.std(x), fmt='o', color='red', alpha=0.8)
 
ax.set_ylabel(r'$'+'('+text1+')-('+text1+')_{fit}$', fontsize=14)
ax.set_xlabel(text2, fontsize=14) 


ax.plot([-2,8], [0,0], 'k:')

X_err_median = np.median(Ec21w)
Y_err_median = np.median(delta*0.+0.1)
Ylm = ax.get_ylim() ; Xlm = ax.get_xlim()
x0 = 0.9*Xlm[0]+0.1*Xlm[1]
y0 = 0.2*Ylm[0]+0.8*Ylm[1]
plt.errorbar([x0], [y0], yerr=[Y_err_median], color='k', fmt='o', alpha=0.7, capsize=3, markersize=5)

xfit = np.linspace(-2,10, 100)
yfit = Fdelta(Delta, xfit)
plt.plot(xfit, yfit, '--', color='blue')

plt.show()

**Wrong**

In [ ]:

from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('axes',edgecolor='k')

data = {'$r-W1$':r_w1, '$Log( W_{mx}^i)$':logWimx, '$c21W1$':c21w, '$\mu 50$':mu50, 'C82': C82}

d = pd.DataFrame.from_dict(data)

sns.set(font_scale=1.2)

pp = sns.pairplot(d, diag_kind="kde", markers=".",plot_kws=dict(s=40, edgecolor="g", linewidth=1),diag_kws=dict(shade=True))

for i in range(3):
  for j in range(3):
    ax = pp.axes[j,i]
    Ylm = ax.get_ylim() ; Xlm = ax.get_xlim()
    add_axis(ax, Xlm, Ylm)
    #ax.grid(color='grey', linestyle='-', linewidth=0.2)

    

In [ ]:
### Compute the correlation matrix
corr = d.corr()

### Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

### Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

### Generate a custom diverging colormap
#cmap = sns.diverging_palette(110, 20, as_cmap=False)

sns.set(font_scale=1.4)

### Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, vmin=-1,vmax=1.0, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .8}, annot_kws={"size": 16}, annot=True) #, mask=mask)




In [ ]:
import sklearn.datasets as ds
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

z_scaler = StandardScaler()

#data = {'$Log( W_{mx}^i)$':logWimx, '$c21W1$':c21w, '$\mu 50$':mu50, 'C82': C82}
data = {'$Log( W_{mx}^i)$':logWimx, '$c21W1$':c21w, '$\mu 50$':mu50}

d = pd.DataFrame.from_dict(data)
z_data = z_scaler.fit_transform(d)

print(z_scaler.scale_, z_scaler.mean_, z_scaler.var_)
## z = (x-u)/s
##u: mean  s:scale  var=s**2


pca_trafo = PCA().fit(z_data);


fig, ax1 = plt.subplots(figsize = (10,6.5))
ax1.plot(pca_trafo.explained_variance_ratio_, '--o', label = 'explained variance ratio');
color =  ax1.lines[0].get_color()
ax1.set_xlabel('principal component', fontsize = 20);
for tl in ax1.get_yticklabels():
    tl.set_color('b')
    
plt.legend(loc=(0.01, 0.075) ,fontsize = 18);

ax1.set_ylim([0,1])

ax2 = ax1.twinx()
ax2.plot(pca_trafo.explained_variance_ratio_.cumsum(), '--go', label = 'cumulative explained variance ratio');
for tl in ax2.get_yticklabels():
    tl.set_color('g')

ax1.tick_params(axis='both', which='major', labelsize=18);
ax1.tick_params(axis='both', which='minor', labelsize=12);
ax2.tick_params(axis='both', which='major', labelsize=18);
ax2.tick_params(axis='both', which='minor', labelsize=12);
plt.xlim([0, 5]);
ax2.set_ylim([0,1])
plt.legend(loc=(0.01, 0),fontsize = 18);

In [ ]:
n_comp =3

pca_data = pca_trafo.fit_transform(z_data)
pca_inv_data = pca_trafo.inverse_transform(np.eye(n_comp))

fig = plt.figure(figsize=(10, 6.5))
sns.heatmap(pca_trafo.inverse_transform(np.eye(n_comp)), cmap="hot", cbar=True, annot=True)
plt.ylabel('principal component', fontsize=20);
plt.xlabel('original feature index', fontsize=20);
plt.tick_params(axis='both', which='major', labelsize=18);
plt.tick_params(axis='both', which='minor', labelsize=12);

fig = plt.figure(figsize=(10, 6.5))
A = pca_trafo.explained_variance_ratio_
#A[2]=0; A[3]=0
j=n_comp-1
while j>1: 
    A[j]=0
    j=j-1
importance =  (np.matmul(A,np.abs(pca_inv_data))) # /(A[0]+A[1])
plt.plot(importance, '--o', label = 'mean')

#pca_trafo.explained_variance_ratio_
#print np.abs(pca_inv_data)
#print pca_trafo.explained_variance_ratio_

#plt.plot(np.square(pca_inv_data.std(axis=0)), '--o', label = 'variance')
plt.legend(loc='lower right')
plt.ylabel('feature contribution', fontsize=20);
plt.xlabel('feature index', fontsize=20);
plt.tick_params(axis='both', which='major', labelsize=18);
plt.tick_params(axis='both', which='minor', labelsize=12);
plt.xlim([0, 5])
plt.legend(loc='lower left', fontsize=18)


###data = {'$Log( W_{mx}^i)$':logWimx, '$c21W1$':c21w, '$\mu 50$':mu50}


In [ ]:
### Compute the correlation matrix
data = PCA().fit_transform(z_data);
d = pd.DataFrame({'pc0':data[:,0],'pc1':data[:,1],'pc2':data[:,2],'$Log( W_{mx}^i)$':logWimx,'$c21W2$':c21w, 'r-w': r_w1, 'mu50':mu50})
sns.set(font_scale=1.2)

pp = sns.pairplot(d, diag_kind="kde", markers=".",plot_kws=dict(s=40, edgecolor="g", linewidth=1),diag_kws=dict(shade=True))



In [ ]:
### Compute the correlation matrix
corr = d.corr()

### Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

### Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

### Generate a custom diverging colormap
#cmap = sns.diverging_palette(110, 20, as_cmap=False)

sns.set(font_scale=1.4)

### Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, vmin=-1,vmax=1.0, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .8}, annot_kws={"size": 16}, annot=True) # , mask=mask)



In [ ]:
pc0 = data[:,0]
pc1 = data[:,1]
pc2 = data[:,2]

############################################# Geting rid of outliers
a0, b0  = np.polyfit(pc0, r_w1, 1)
delta = np.abs(r_w1-(a0*pc0+b0))
indx = np.where(delta<0.5)
r_w1 = r_w1[indx]
pc0 = pc0[indx]
pc1 = pc1[indx]
pc2 = pc2[indx]
#############################################

plt.plot(r_w1,pc0, 'g.')
a0, b0  = np.polyfit(pc0, r_w1, 1)
y = np.linspace(-5,5,50)
x = a0*y+b0
plt.plot(x,y, 'r--')
plt.xlim([-2,2])

plt.xlabel('Color')
plt.ylabel('$PC0$')


# from sklearn.linear_model import LinearRegression
# lin=LinearRegression()
# r_w1_ = r_w1[:, np.newaxis]
# lin.fit(r_w1_,pc0)
# x = np.linspace(-1,2,50)
# x=x[:, np.newaxis]
# y=lin.predict(x)
# plt.plot(x,y, 'b--')




In [ ]:
delta = r_w1-(a0*pc0+b0)
pc = pc1

rms = np.sqrt(np.mean(np.square(delta)))
print(rms)   # 0.157


plt.plot(pc, delta, 'k.')
a, b, c  = np.polyfit(pc, delta, 2)
x = np.linspace(-10,10,50)
y = a*x**2+b*x+c
plt.plot(x,y, 'g--')
plt.xlim([-3,3])
plt.ylim([-0.7,0.7])

plt.xlabel('$PC1$')
plt.ylabel('$\Delta$') 

for i in np.arange(-2,2,0.5):
    
    x = []
    y = []
    for ii in range(len(pc)):
        xi = pc[ii]
        if xi>=i and xi<i+0.5:
            x.append(xi)
            y.append(delta[ii])
    if len(x)>0:
        plt.errorbar(np.median(x), np.median(y), yerr=np.std(y), xerr=np.std(x), fmt='o', color='red', alpha=0.8)

In [ ]:
delta = r_w1-(a0*pc0+b0)
pc = pc2

plt.plot(pc, delta, 'k.')
a, b, c  = np.polyfit(pc, delta, 2)
x = np.linspace(-10,10,50)
y = a*x**2+b*x+c
plt.plot(x,y, 'g--')
plt.xlim([-3,3])
plt.ylim([-0.7,0.7])

plt.xlabel('$PC2$')
plt.ylabel('$\Delta$')

for i in np.arange(-3,3,0.5):
    
    x = []
    y = []
    for ii in range(len(pc)):
        xi = pc[ii]
        if xi>=i and xi<i+0.5:
            x.append(xi)
            y.append(delta[ii])
    if len(x)>0:
        plt.errorbar(np.median(x), np.median(y), yerr=np.std(y), xerr=np.std(x), fmt='o', color='red', alpha=0.8)